##LangChain Index について

##インデックスとは
インデックスは既存のドキュメントと LLM を組合わせて使用するのに役立ちます。 インデックスを使うことで、ドキュメントを構造化し、LLM がドキュメントと最適なやり取りができるように調整できます。

インデックスのもっとも一般的な使用例は「検索（retrieval）」の実装です。 ユーザーのクエリから、もっとも関連性の高いドキュメントを返すようなタスクの実行に利用できます。 さらに、チェーン内で利用すれば、ユーザーからの質問に対して渡したドキュメントを基準に答えるようなタスクも実行できます。

###インデックスと BaseRetriever
LangChain では、以下に示す BaseRetriever クラスを利用することを目標にインデックスを構築していきます。



In [1]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.7 MB/s eta 0:00:00


In [2]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.8 MB/s eta 0:00:00


In [3]:
pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32001 sha256=3150016fa0162507b177bd4abe170d752c50ce665e9fc963d16b2a5661286f98
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [4]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.8 MB/s eta 0:00:00


In [5]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv(OPENAI_API_KEY)
os.environ["SERPAPI_API_KEY"] =os.getenv(SERPAPI_API_KEY)

In [6]:
from langchain.llms import OpenAI
from abc import ABC, abstractmethod
from typing import List
from langchain.schema import Document

In [ ]:
class BaseRetriever(ABC):
    @abstractmethod
    def get_relevant_documents(self, query: str) -> List[Document]:
        """Get texts relevant for a query.

        Args:
            query: string to find relevant tests for

        Returns:
            List of relevant documents
        """

このクラスは、get_relevant_documents というメソッドを持ち、これは受け取ったクエリから関連するドキュメントを返すようなメソッドです。 つまり、このクラスのオブジェクトを構成さえすれば、検索を実行できるようになります。

###インデックスを構成する機能

インデックスモジュールはおもに以下の４つの機能から構成されます。

- Document Loaders: さまざまなソースからドキュメントを読み込むのに使います
- Text Splitters: テキスト分割の実行に使います
- VectorStores: ベクターストアを利用します
- Retrievers: ドキュメントと言語モデルの結合を容易にする汎用的なインターフェースです

以下で、これらの機能の概要と使用例を示します。

LangChain はデフォルトで Chroma を VectorStore として使用します。 この節では、Chroma の使用例として、txt ファイルを読み込み、そのテキストに関する質問応答をする機能を構築します。

まずはじめに chromadb をインストールしてください。

In [7]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.0/399.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━

テキストに対する質問応答は、以下の4つのステップで構成されます。

1. インデックスの作成
2. そのインデックスから Retriever を作成
3. 質問回答チェーンを作成
4. 機能の利用（質問を入力し、回答を取得）

後ほどこれらの機能を説明しますが、その前に、これらをより簡単に利用する方法を紹介します。

###VectorstoreIndexCreator によるインデックスの作成と質問の実行

LangChain では、VectorstoreIndexCreator を利用することで、簡単にインデックスを作成できます。 さらに、このクラスを用いて作成される VectorStoreIndexWrapper オブジェクトには、query というメソッドが用意されており、簡単に質問と回答の取得ができます。


まずは、インデックスを作成します。 ここでは、一例として、夏目漱石の『[坊ちゃん](https://www.aozora.gr.jp/cards/000148/card752.html)』 を読みこんでインデックスを作成します。

In [8]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/data/Random/functions.txt"


Mounted at /content/drive


In [9]:
file1 = open(path, "r",  encoding="Shift-JIS")


In [10]:
# ドキュメントローダーをインポート
from langchain.document_loaders import TextLoader
# ドキュメントローダーの初期化
loader = TextLoader(file_path=path,  autodetect_encoding =True)
# インデックスの作成に用いるクラスをインポート
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper


In [11]:
# ベクターストアの作成
index: VectorStoreIndexWrapper = VectorstoreIndexCreator().from_loaders([loader])

これで、インデックスのラッパーが得られました。 ためしに、この VectorStoreIndexWrapper を利用して質問を実行してみましょう。 上記のコードに以下のようなコードを追加してください。



In [13]:
order = 'encrypt pass word varible in my .env file'
query = "I would like to {}, which function name is matched according to function descriptions?".format(order)
answer = index.query(query)
print('Question: ', query)
print('Answer: ', answer)

Question:  I would like to encrypt pass word varible in my .env file, which function name is matched according to function descriptions?
Answer:   encrypt_data


たったこれだけのコードで、インデックスの作成から、質問の実行までができました。


また、VectorStoreIndexWrapper オブジェクトから、BaseRetriever オブジェクトを作成することもできます。 さらに、作成した BaseRetriever の get_relevant_documents メソッドを利用すると、質問に関連するドキュメントを取得可能です。

In [ ]:
retriever = index.vectorstore.as_retriever()

docs = retriever.get_relevant_documents(query=query)
for doc in docs:
  print(doc)

page_content='「つまりどっちがいいんですかね」\n「つまり月給の多い方が豪《えら》いのじゃろうがなもし」' metadata={'source': '/content/drive/My Drive/data/Random/bocchan.txt'}
page_content='「じゃ何と云うんだ」\n「ハイカラ野郎の、ペテン師の、イカサマ師の、猫被《ねこっかぶ》りの、香具師《やし》の、モモンガーの、岡っ引きの、わんわん鳴けば犬も同然な奴とでも云うがいい」\n「おれには、そう舌は廻らない。君は能弁だ。第一単語を大変たくさん知ってる。それで演舌《えんぜつ》が出来ないのは不思議だ」\n「なにこれは喧嘩《けんか》のときに使おうと思って、用心のために取っておく言葉さ。演舌となっちゃ、こうは出ない」\n「そうかな、しかしぺらぺら出るぜ。もう一遍やって見たまえ」\n「何遍でもやるさいいか。――ハイカラ野郎のペテン師の、イカサマ師の……」と云いかけていると、椽側《えんがわ》をどたばた云わして、二人ばかり、よろよろしながら馳《か》け出して来た。\n「両君そりゃひどい、――逃げるなんて、――僕が居るうちは決して逃《にが》さない、さあのみたまえ。――いかさま師？――面白い、いかさま面白い。――さあ飲みたまえ」\nとおれと山嵐をぐいぐい引っ張って行く。実はこの両人共便所に来たのだが、酔《よ》ってるもんだから、便所へはいるのを忘れて、おれ等を引っ張るのだろう。酔っ払いは目の中《あた》る所へ用事を拵えて、前の事はすぐ忘れてしまうんだろう。\n「さあ、諸君、いかさま師を引っ張って来た。さあ飲ましてくれたまえ。いかさま師をうんと云うほど、酔わしてくれたまえ。君逃げちゃいかん」\nと逃げもせぬ、おれを壁際《かべぎわ》へ圧《お》し付けた。諸方を見廻してみると、膳の上に満足な肴の乗っているのは一つもない。自分の分を奇麗《きれい》に食い尽《つく》して、五六間先へ遠征《えんせい》に出た奴もいる。校長はいつ帰ったか姿が見えない。' metadata={'source': '/content/drive/My Drive/data/Random/bocchan.txt'}
page_content='「古賀さんは、だってここの人じゃありませんか」\n「ここの地《じ》の人ですが、少し都合があって――

##インデックスの作成
先の節では、VectorstoreIndexCreator を利用して、簡単にインデックスを作成する方法を紹介しました。

この節からは VectorstoreIndexCreator がどのようにインデックスを作成し、利用しているのかを理解するために、以下のステップを順に実行するプログラム例を示します。 VectorstoreIndexCreator を使うと、下記の手順のうち、2. ~ 4. のステップを簡単に実行できます

先の節では、VectorstoreIndexCreator を利用して、簡単にインデックスを作成する方法を紹介しました。

この節からは VectorstoreIndexCreator がどのようにインデックスを作成し、利用しているのかを理解するために、以下のステップを順に実行するプログラム例を示します。 VectorstoreIndexCreator を使うと、下記の手順のうち、2. ~ 4. のステップを簡単に実行できます

1. ドキュメントの読みこみ

In [ ]:
# テキストローダーをインポート
from langchain.document_loaders import TextLoader

# テキストローダーの初期化
loader = TextLoader(file_path=path,  autodetect_encoding =True)

# ドキュメントの読みこみ
documents = loader.load()

2. ドキュメントを「チャンク」に分割

In [ ]:
# チャンクサイズの制限を下回るまで再帰的に分割するテキストスプリッターのインポート
from langchain.text_splitter import RecursiveCharacterTextSplitter

# テキストスプリッターの初期化
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=0)

# テキストをチャンクに分割
texts = text_splitter.split_documents(documents)

3. 各ドキュメントに対してエンベッディングを用意

In [ ]:
# 使用するエンベッディングをインポート
from langchain.embeddings import OpenAIEmbeddings

# エンベッディングの初期化
embeddings = OpenAIEmbeddings()

4. ドキュメントとエンベッディングをベクターストアに格納

インデックスとして用いるベクターストアを作成します。



In [ ]:
# vectorstore をインポート (ここでは Chroma を使用)
from langchain.vectorstores import Chroma

# ベクターストアにドキュメントとエンベッディングを格納
db = Chroma.from_documents(texts, embeddings)

5. インデックスから Retriever を作成

上記の手順で作成したベクターストアをインデックスとして利用して、質問に対する回答を取得します。 そのためにまず、BaseRetriever オブジェクトを作成します

In [ ]:
retriever = db.as_retriever()

6. 質問の実行

これで BaseRetriever オブジェクトを作成できました。 あとは質問をするだけです。 しかし、VectorStoreIndexCreator を使って VectorStoreIndexWrapper を作成した場合とは異なり、BaseRetriever では質問を実行するためのメソッドが用意されていません。

そこで、以下のようにチェーンを作成して利用します。

In [ ]:
# OpenAI を使うためのインポート
from langchain.llms import OpenAI

# LLM ラッパーの初期化
llm = OpenAI(model_name="text-davinci-003", temperature=0, max_tokens=500)

# 質問と回答の取得に使用するチェーンをインポート
from langchain.chains import RetrievalQA

# チェーンを作り、それを使って質問に答える
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

query = "主人公の職業は？"
answer = qa.run(query)

In [ ]:
print(answer)

 教頭
